In [325]:
# Imports
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sqlalchemy import create_engine
import warnings

In [326]:
# Load data
path = 'aki_patients_filtered'
data = pd.read_csv(path + '.csv')
# data = data.head(1000)

In [327]:
# Create basic format and add creatinine values
def creatinine(ICU_stay, timeframe=7):
    group = ICU_stay[1]
    
    # Turn charttime into datetime
    group['charttime'] = pd.to_datetime(group['charttime'], format='%Y-%m-%d %H:%M:%S')
    group = group[['subject_id', 'stay_id', 'charttime', 'valuenum', 'worsened']]

    # Save latest date
    latest_date = group['charttime'].max()

    # Create new datetimes
    group.set_index('charttime', inplace=True)
    new_index = pd.date_range(end=group.index.max().floor('D'), periods=timeframe, freq='D')
    new_group = pd.DataFrame(index=new_index)
    group = pd.concat([group, new_group], axis=0).sort_index()

    # Fill in missing values and remove old datetimes
    group['valuenum'] = group['valuenum'].bfill()
    group = group[group['subject_id'].isna()]

    # Clean up
    group['date'] = [f"creatinine_{i}" for i in range(1, timeframe+1)]
    group.reset_index(drop=True,inplace=True)

    # Add subject id
    group['subject_id'] = ICU_stay[1]['subject_id'].iloc[0]

    # Pivot the table based on subject_id and dates
    group = group.pivot_table(index=['subject_id'], columns='date', values='valuenum', aggfunc='first')
    group.columns.name = None

    # Insert columns
    group.insert(0, 'last_day', latest_date)
    group.insert(0, 'stay_id', ICU_stay[1]['stay_id'].iloc[0])
    group.insert(0, 'subject_id', ICU_stay[1]['subject_id'].iloc[0])
    group.insert(0, 'label', ICU_stay[1]['worsened'].iloc[0])

    return group

In [328]:
# Connect to SQL database
account = 'andrew'
password = 'andrew0613'
ip = '140.96.146.59'
port = "15432"
db_name = 'mimic'
engine = create_engine(f'postgresql+psycopg2://{account}:{password}@{ip}:{port}/{db_name}')
conn = engine.connect()

def sql_query(text):
    sql_df = pd.read_sql_query(text, conn)
    return sql_df

In [329]:
# Add static data
def static(ICU_stay, sample):
    group = ICU_stay[1]
    subject_id = group['subject_id'].iloc[0]

    # Select and insert patient data
    text = f"""
    SELECT *
    FROM andrew.patients_static
    WHERE subject_id = {subject_id}
    """
    patient_data = sql_query(text)
    if patient_data.empty: return sample

    patient_data.set_index('subject_id', inplace=True)
    patient_data.drop(columns=['dod'], inplace=True)
    
    return pd.concat([sample, patient_data], axis=1)

In [330]:
# Add sequential data
def sequential(ICU_stay, sample, timeframe):
    group = ICU_stay[1]
    stay_id = group['stay_id'].iloc[0]

    query = f"""
        SELECT *
        FROM andrew.chartevents
        WHERE stay_id = {stay_id} AND charttime <= '{sample['last_day'].iloc[0]}'
        ORDER BY itemid, charttime DESC
        """
    chartevents = sql_query(query)

    chartevents_formatted = pd.DataFrame()
    for group in chartevents.groupby('itemid'):
        group = resample_and_interpolate(group[1], timeframe)
        chartevents_formatted = pd.concat([chartevents_formatted, group], axis=1)
    return pd.concat([sample, chartevents_formatted], axis=1)

# Helper function to resample and interpolate
def resample_and_interpolate(group, timeframe):
    itemid = group['itemid'].iloc[0]
    subjectid = group['subject_id'].iloc[0]

    # Create new datetimes
    group.set_index('charttime', inplace=True)
    new_index = pd.date_range(end=group.index.max().floor('D'), periods=timeframe, freq='D')
    new_group = pd.DataFrame(index=new_index)
    group = pd.concat([group, new_group], axis=0).sort_index()

    # Use interpolation to fill in missing values and bfill for last values
    group['valuenum'] = group['valuenum'].interpolate(method='time').bfill()
    group = group[group['subject_id'].isna()]


    # Clean up
    group['date'] = [f"{itemid}_{i}" for i in range(1, timeframe+1)]
    group.reset_index(inplace=True)

    # Add subject id
    group['subject_id'] = subjectid

    # Pivot the table based on subject_id and dates
    group = group.pivot_table(index=['subject_id'], columns='date', values='valuenum', aggfunc='first')
    group.columns.name = None

    return group

In [331]:
df = pd.DataFrame()
timeframe = 7
i = 0
for ICU_stay in data.groupby('stay_id'):
    sample = creatinine(ICU_stay, timeframe)
    sample = static(ICU_stay, sample)
    sample = sequential(ICU_stay, sample, timeframe)
    df = pd.concat([df, sample])
    # i+=1
    # if i == 40: break
df

,label,subject_id,stay_id,last_day,creatinine_1,creatinine_2,creatinine_3,creatinine_4,creatinine_5,creatinine_6,...,227024_5,227024_6,227024_7,227054_1,227054_2,227054_3,227054_4,227054_5,227054_6,227054_7
subject_id,,,,,,,,,,,,,,,,,,,,,
10682002,0,10682002,30003087,2132-12-06 04:43:00,0.7,0.7,0.7,0.7,0.7,0.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16165135,0,16165135,30003125,2116-04-09 02:12:00,0.7,0.6,0.6,0.6,0.6,0.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16235911,1,16235911,30003306,2188-06-07 02:54:00,3.1,3.1,3.1,3.1,3.1,3.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10332722,0,10332722,30005362,2157-01-02 04:45:00,2.9,2.9,2.9,3.0,3.1,3.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18756985,1,18756985,30006983,2159-11-08 04:45:00,1.7,1.6,1.6,1.6,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15530538,1,15530538,39986775,2123-10-09 03:14:00,0.9,0.9,0.9,0.9,0.9,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11414090,0,11414090,39987151,2170-05-11 04:51:00,0.7,0.7,0.6,0.7,0.6,0.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12719159,1,12719159,39989733,2120-09-07 04:50:00,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [339]:
print(df.isna().sum().sort_values(ascending=True))
print(df['label'].value_counts())

label          0
220615_5       0
220615_4       0
220615_3       0
220615_2       0
            ... 
227010_5    5426
227010_7    5426
227023_1    5426
227023_3    5426
227054_7    5426
Length: 361, dtype: int64
label
0    2733
1    2694
Name: count, dtype: int64


In [382]:
# Normalize all columns
def normalize(data):
    scaler = StandardScaler()
    data.iloc[:, 4:] = scaler.fit_transform(data.iloc[:, 4:])
    return data
temp = df.copy()
temp = temp.dropna(axis=1, thresh=0.99*len(temp))
temp = temp.fillna(temp.mean())
temp = normalize(temp)

# Drop duplicate creatinine columns
temp = temp.drop(columns=['220615_1', '220615_2', '220615_3', '220615_4', '220615_5', '220615_6', '220615_7'])

temp.to_csv(path + '_clean.csv', index=False)
temp.shape

(5427, 67)

In [335]:
# df.to_csv(path + '_all_cols_clean.csv', index=False)